Step 0: Make sure you have the data in the formats specified by the README.

Step 1: Copy the audios from sample directory and sort them into the "audio" folder by levels based on your CSV file

In [17]:
!python split_by_rating.py

Unique values in 'Fluency': ['A2' 'B1' 'B2' 'C1' 'A1' 'C2']
Unique values in 'Fluency2': ['C1' 'B1' 'A2' 'C2' 'B2' 'B3' 'A1']
Rows with 'Unknown' merged fluency after rounding:
Empty DataFrame
Columns: [Audio, Fluency, Fluency2, Fluency_Numeric, Fluency2_Numeric, Merged_Fluency_Numeric, Merged_Fluency]
Index: []
Processed fluency ratings saved to: audio/processed_audio_fluency.csv
Copied: sampled_audio\audio_33.wav -> audio\C1
Copied: sampled_audio\audio_92.wav -> audio\B1
Copied: sampled_audio\audio_100.wav -> audio\B1
Copied: sampled_audio\audio_112.wav -> audio\A2
Copied: sampled_audio\audio_137.wav -> audio\C2
Copied: sampled_audio\audio_155.wav -> audio\C1
Copied: sampled_audio\audio_180.wav -> audio\C2
Copied: sampled_audio\audio_236.wav -> audio\B2
Copied: sampled_audio\audio_243.wav -> audio\B2
Copied: sampled_audio\audio_260.wav -> audio\C1
Copied: sampled_audio\audio_296.wav -> audio\A2
Copied: sampled_audio\audio_374.wav -> audio\B1
Copied: sampled_audio\audio_402.wav -> aud

In [2]:
# Creates a CSV file listing all the audios in a validation directory (Optional)

!python create_wav_list.py

CSV file 'validation_audio/audio.csv' created successfully.


Step 2: Slice (optional). This *might* help the classifier performance if you have long audios (> 30 seconds) but it is up to you to experiment with this one. You will need to edit the paths in preprocess.py and run_praat.py.

In [24]:
!python slice.py


  0%|          | 0/280 [00:00<?, ?it/s]
concurrent.futures.process._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\multiprocessing\queues.py", line 244, in _feed
    obj = _ForkingPickler.dumps(obj)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\multiprocessing\reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
AttributeError: Can't pickle local object 'process_audio_files_in_parallel.<locals>.<lambda>'
"""

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\zrack\fluency_classification\slice.py", line 119, in <module>
    processed_data = process_audio_files_in_parallel(audio_files_csv, output_base_dir)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Step 3: Preprocess

In [7]:
!python preprocess.py

Sliced folder not found: audio\A1
Sliced folder not found: audio\A2
Sliced folder not found: audio\B1
Sliced folder not found: audio\B2
Sliced folder not found: audio\C1
Sliced folder not found: audio\C2
Processing files in: audio\Unknown
Completed processing for: audio\Unknown
All files have been preprocessed, normalized, filtered, and replaced.


Step 4: Calculate praat scores. If there is a praat error, it will stop the rest of the folder from processing. Sometimes if you try again, it'll work. But other times it's easiest to just delete the problematic audio file. It's usually a problem with undefined pitch ceiling. Expect this cell to take several minutes to run. Warnings are fine, but Formula not run means it terminated early for the whole folder.

In [8]:
#Delete previous SyllableNuclei files

import os

for file in os.listdir("."):  # List all files in the current directory
    if file.startswith("SyllableNuclei") and file.endswith(".txt"):
        try:
            os.remove(file)  # Delete the file
            print(f"Deleted: {file}")
        except Exception as e:
            print(f"Error deleting {file}: {e}")

In [9]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [12]:
!python run_praat.py

[DEBUG] Sliced folder not found: audio\A1
[DEBUG] Sliced folder not found: audio\A2
[DEBUG] Sliced folder not found: audio\B1
[DEBUG] Sliced folder not found: audio\B2
[DEBUG] Sliced folder not found: audio\C1
[DEBUG] Sliced folder not found: audio\C2
[DEBUG] Processing all files in folder: audio\Unknown
[DEBUG] Running command: C:/Program Files/Praat/praat.exe --run Nuclei.praat audio\Unknown\*.wav Band pass (300..3300 Hz) -25.0 2.0 0.3 yes English 1.0 Save as text file OverWriteData yes
[Praat stdout]: W a r n i n g :   r e p l a c e d   1 0 / 5 6   F 0   v a l u e s   b y   m e a n   ( 7 . 8 9 5 )   i n   a u d i o _ 2 1 9 3 . 
[Praat stdout]:  
[Praat stdout]:  W a r n i n g :   r e p l a c e d   1 0 / 5 6   d F 0   v a l u e s   b y   m e a n   ( 2 . 7 8 7 )   i n   a u d i o _ 2 1 9 3 . 
[Praat stdout]:  
[Praat stdout]:  W a r n i n g :   r e p l a c e d   1 0 / 5 6   d q F 0   v a l u e s   b y   m e a n   ( 7 . 9 0 2 )   i n   a u d i o _ 2 1 9 3 . 
[Praat stdout]:  
[Praat st

In [13]:
!python combine_praat.py

Number of unmatched rows: 0
Sample of unmatched rows:
Empty DataFrame
Columns: [name,  nsyll,  npause,  dur(s),  phonationtime(s),  speechrate(nsyll/dur),  articulation_rate(nsyll/phonationtime),  ASD(speakingtime/nsyll),  nrFP,  tFP(s), fluency]
Index: []
Merged data has been saved to audio/totrain.csv


In [35]:
# Optional, doesn't work very well. Random forest features doesn't include the pronunciation info here, but you can make add in the columns.
# Requires an openai key

!python pronunciation.py

worked for sound audio\A1\audio_1918.wav
worked for sound audio\A1\Audio 95.wav
worked for sound audio\A1\audio_1919.wav
worked for sound audio\A1\audio_1206.wav
worked for sound audio\A2\audio_1668.wav
worked for sound audio\A2\Audio 87.wav
worked for sound audio\A1\audio_1980.wav
worked for sound audio\A2\audio_1951.wav
worked for sound audio\A2\audio_2133.wav
worked for sound audio\A1\audio_1945.wav
worked for sound audio\A2\Audio 28.wav
worked for sound audio\A2\audio_1674.wav
worked for sound audio\A1\audio_1209.wav
worked for sound audio\A2\audio_296.wav
worked for sound audio\A1\audio_1949.wav
worked for sound audio\A2\audio_1957.wav
worked for sound audio\A1\audio_1981.wav
worked for sound audio\A2\audio_519.wav
worked for sound audio\A2\Audio 91.wav
worked for sound audio\A1\Audio 33.wav
worked for sound audio\A2\audio_1675.wav
worked for sound audio\A1\Audio 39.wav
worked for sound audio\A2\audio_521.wav
worked for sound audio\A2\audio_1958.wav
worked for sound audio\A2\Audio

In [2]:
# This merges the temporal acoustic feature columns with the pronunciation feature columns. Don't worry about this if you haven't run pronunciation.py

import pandas as pd

# Load the first CSV file
df1 = pd.read_csv('audio/audio.csv')

# Load the second CSV file with encoding
df2 = pd.read_csv('audio/processed_results2.csv', encoding='utf-8')

# # Remove ".wav" from the 'name' column in df2
# df2['name'] = df2['name'].str.replace(' ', '_')
# df2['name'] = df2['name'].str.replace('.wav', '', regex=False)

# Merge the two dataframes on the common column ('name')
merged_df = pd.merge(df1, df2, on='name')

# Save the merged dataframe to a new CSV file
merged_df.to_csv('audio/merged_output.csv', index=False)

print("Merged CSV saved as 'merged_output.csv'")


Merged CSV saved as 'merged_output.csv'


Step 5: Train and Evaluate RandomForest

In [12]:
%matplotlib inline

!python train_forest.py

^C


In [ ]:
%matplotlib inline

!python train_experimental.py

  File "c:\Users\zrack\fluency_classification\train_experimental.py", line 253
    print(f"Cohen's Kappa: {res['Cohen\'s Kappa']:.4f}")
                                                       ^
SyntaxError: f-string expression part cannot include a backslash
